Neural Machine Translation 
- Many-to-Many
Data set
- original : http://www.statmt.org/wmt14/translation-task.html

In [1]:
!pip3 install torch torchvision

    100% |████████████████████████████████| 519.5MB 31kB/s 
tcmalloc: large alloc 1073750016 bytes == 0x58676000 @  0x7f94393e31c4 0x46d6a4 0x5fcbcc 0x4c494d 0x54f3c4 0x553aaf 0x54e4c8 0x54f4f6 0x553aaf 0x54efc1 0x54f24d 0x553aaf 0x54efc1 0x54f24d 0x553aaf 0x54efc1 0x54f24d 0x551ee0 0x54e4c8 0x54f4f6 0x553aaf 0x54efc1 0x54f24d 0x551ee0 0x54efc1 0x54f24d 0x551ee0 0x54e4c8 0x54f4f6 0x553aaf 0x54e4c8
    100% |████████████████████████████████| 61kB 2.2MB/s 
    100% |████████████████████████████████| 2.0MB 2.1MB/s 
  Found existing installation: Pillow 4.0.0
    Uninstalling Pillow-4.0.0:
      Successfully uninstalled Pillow-4.0.0


# **데이터 다운로드**

In [3]:
# 10만개의 영어 문장 데이터 다운로드
!wget https://goo.gl/DBcpZg -O en.txt
# 10만개 프랑스 문장 데이터 다운로드
!wget https://goo.gl/AuBFga -O fr.txt
!ls

--2018-08-17 01:16:07--  https://goo.gl/DBcpZg
Resolving goo.gl (goo.gl)... 74.125.201.102, 74.125.201.100, 74.125.201.139, ...
Connecting to goo.gl (goo.gl)|74.125.201.102|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://davian.korea.ac.kr/filemanager/wl/?id=tEOi7IVh1ICcSe38jjNPTGAcAotwEzp2 [following]
--2018-08-17 01:16:07--  http://davian.korea.ac.kr/filemanager/wl/?id=tEOi7IVh1ICcSe38jjNPTGAcAotwEzp2
Resolving davian.korea.ac.kr (davian.korea.ac.kr)... 163.152.29.74
Connecting to davian.korea.ac.kr (davian.korea.ac.kr)|163.152.29.74|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7186902 (6.9M) [application/octet-stream]
Saving to: ‘en.txt’

en.txt              100%[===================>]   6.85M   329KB/s    in 23s     

2018-08-17 01:16:34 (308 KB/s) - ‘en.txt’ saved [7186902/7186902]

--2018-08-17 01:16:35--  https://goo.gl/AuBFga
Resolving goo.gl (goo.gl)... 108.177.112.100, 108.177.112.102, 108.177.112.13

# Split train / test set 
- validation set : lr , hidden layer 개수 등 validation set 을 이용하여 정함 

In [6]:
src_lines = open("en.txt", "rt", encoding="utf8").readlines()[:50000]
trg_lines = open("fr.txt", "rt", encoding="utf8").readlines()[:50000]

from sklearn.model_selection import train_test_split

src_train, src_test, trg_train, trg_test = train_test_split(src_lines, trg_lines, test_size=3000, random_state=42)
print("train size: {:,} test size: {:,}".format(len(src_train), len(src_test)))


train size: 47,000 test size: 3,000


# 단어장 만들기

In [11]:
from collections import Counter
# 영어
src_w2i = {
    '<PAD>':0,
    '<UNK>':1,
    '<SOS>':2,
    '<EOS>':3
}
# 프랑스어
trg_w2i = {
    '<PAD>':0,
    '<UNK>':1,
    '<SOS>':2,
    '<EOS>':3
}
# 단어 개수
src_counter = Counter()
trg_counter = Counter()
for src, trg in zip(src_train, trg_train):
  src_counter.update(src.split(" "))
  trg_counter.update(trg.split(" "))

# word to index
max_len = 10000 # 최대 활용할 단어 갯수
for index, (word, cnt) in enumerate(src_counter.most_common(max_len)):
  src_w2i.update({word: index + 4})

for index, (word, cnt) in enumerate(trg_counter.most_common(max_len)):
  trg_w2i.update({word: index + 4})  

# index to word 
src_i2w = dict(zip(src_w2i.values(), src_w2i.keys()))
trg_i2w = dict(zip(trg_w2i.values(), trg_w2i.keys()))

print(list(src_w2i.items())[:10])
print(list(src_i2w.items())[:10])
print()
print(list(trg_w2i.items())[:10])
print(list(trg_i2w.items())[:10])

[('<PAD>', 0), ('<UNK>', 1), ('<SOS>', 2), ('<EOS>', 3), ('the', 4), ('to', 5), ('of', 6), ('is', 7), ('in', 8), ('a', 9)]
[(0, '<PAD>'), (1, '<UNK>'), (2, '<SOS>'), (3, '<EOS>'), (4, 'the'), (5, 'to'), (6, 'of'), (7, 'is'), (8, 'in'), (9, 'a')]

[('<PAD>', 0), ('<UNK>', 1), ('<SOS>', 2), ('<EOS>', 3), ('de', 4), ('la', 5), ('le', 6), ('à', 7), ('les', 8), ('des', 9)]
[(0, '<PAD>'), (1, '<UNK>'), (2, '<SOS>'), (3, '<EOS>'), (4, 'de'), (5, 'la'), (6, 'le'), (7, 'à'), (8, 'les'), (9, 'des')]


# Dataloader

In [0]:
from torch.utils.data import Dataset, DataLoader 
# 숫자가 주어지면 단어가 나오는 클래스
class SequenceData(Dataset):
  def __init__(self, src_lines, trg_lines):
    super(SequenceData, self).__init__()
    self.dataset = list(zip(src_lines, trg_lines))
    
  def __len__(self):
    return len(self.dataset)
  
 
  def __getitem__(self, item):
    src_sent, trg_sent = self.dataset[item]
    src_vec, trg_vec =[], []
    
    for word in src_sent.strip().split(" "):
      vector = src_w2i[word] if src_w2i.get(word, False) else src_w2i['<UNK>']
      src_vec.append(vector)
      
    for word in trg_sent.strip().split(" "):
      vector = trg_w2i[word] if trg_w2i.get(word, False) else trg_w2i['<UNK>']
      trg_vec.append(vector)
    trg_vec.append(trg_w2i['<EOS>'])
    
    
    return src_vec, trg_vec
  
def collate_fn(data):
  """
  [['영어1', '영어2','영어3'], ['프랑스어1', '프랑스2', '프랑스3']]
  [['영어1', '프랑스1'], [영어2, 프랑스어2], [영어3, 프랑스어3]]
  """
  batch_src, batch_trg = zip(*data)
  
  src_max_len = max([len(s) for s in batch_src])
  trg_max_len = max([len(s) for s in batch_trg])
  
  for s, t in zip(batch_src, batch_trg):
    s += [src_w2i['<PAD>']] * (src_max_len - len(s)) # 문장에 padding 을 붙여줌. 문장의 길이가 달라서 따로 처리해야되는데 padding 이 있으면 알아서 처리
    t += [trg_w2i['<PAD>']] * (trg_max_len - len(s))
    
  return batch_src, batch_trg

In [23]:
dataset = SequenceData(src_train, trg_train)
dataloader = DataLoader(dataset, 4, shuffle=False, num_workers=0, collate_fn=collate_fn)

for i, (src, trg) in enumerate(dataloader): # enumerate : return 값이 쪼개짐
  if i == 1:
    break
    
    
  for s, t in zip(src, trg):
    print(len(s))
    print([src_i2w[v] for v in s])
    print(len(t))
    print([trg_i2w[v] for v in t])
    print()

19
['Commissioner', 'Barnier,', 'you', 'laid', 'great', 'stress', 'in', 'your', 'statement', 'on', 'the', 'importance', 'of', 'dialogue', 'with', 'the', 'citizens', 'of', '<UNK>']
21
['Monsieur', 'le', 'Commissaire', 'Barnier,', 'vous', 'avez', 'fortement', '<UNK>', 'dans', 'votre', 'déclaration,', 'sur', "l'importance", 'du', 'dialogue', 'avec', 'les', 'citoyens', '<UNK>', '<EOS>', '<PAD>']

19
['He', 'wants', 'to', 'be', 'the', '<UNK>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>']
7
['Il', 'veut', 'être', 'lui-même', '<UNK>', '<EOS>', '<PAD>']

19
['Commissioner', "Kinnock'", 's', 'package', 'of', 'reforms', 'must', 'be', 'pushed', 'through', 'without', '<UNK>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>']
15
['<UNK>', 'de', 'réformes', 'proposé', 'par', 'le', 'commissaire', 'Kinnock,', 'doit', 'être', 'adoptée', 'sans', '<UNK>', '<EOS>', '<PAD>']

19
['On', 'the', 'contrary,', 'Ireland', 'is', 

# Encoder model
- https://pytorch.org/docs/stable/nn.html#gru
- RNN(input, hidden, size)

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [0]:
class Encoder(nn.Module):
  def __init__(self, vocab_size, embed_size, hidden_size, num_layers, bidirectional=False):
    super(Encoder, self).__init__()
    self.embedding = nn.Embedding(vocab_size, embed_size, padding_idx=src_w2i['<PAD>']) # word를 embedding
    self.rnn = rnn.GRU(embed_size, hidden_size, num_layers, batch_first=True)
    
  def forward(self, source, h):
    """
    source : source language sentences
    h : initial hidden state
    """
    
    #embed_input_size 
    embed_input = self.embedding(source)
    
    
    encode_output, last_hidden_state = self.rnn(embed_input, h) #embedding 된 단어 vector를 rnn에 넣어줌. 
    return encode_output, last_hidden_state

In [33]:
print(src)
src = torch.tensor(src, dtype=torch.long)
print(src.shape)

encoder = Encoder(10004, 100, 200, 1, False)
h_0 = torch.zeros(1, src.shape[0], 200)
output, h_t = encoder(src, h_0)

print(output.shape)
print(h_t.shape)

tensor([[   1,    6, 1875,    1,    1,    8, 7485,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0],
        [  11,   37,   49,    5, 2838,   12,   18,   17,   53, 2175,    5, 1326,
         2114,   10, 9462,    4,    1],
        [ 398,   52, 2744,    5,    4, 1991,    6,    4, 9463,   54,    1,    0,
            0,    0,    0,    0,    0],
        [  15,   80,    7,    1,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0]])
torch.Size([4, 17])
torch.Size([4, 17, 200])
torch.Size([1, 4, 200])


In [0]:
class Decoder(nn.Module):
   def __init__(self, vocab_size, embed_size, hidden_size, num_layers, bidirectional=False):
    super(Decoder, self).__init__()
    self.embedding = nn.Embedding(vocab_size, embed_size, padding_idx=trg_w2i['<PAD>']) # word를 embedding
    self.rnn = rnn.GRU(embed_size, hidden_size, num_layers, batch_first=True)
   
    def forward(self, target, h):
      
      """
      target : target language sentences
      h : initial hidden state
      """

      #embed_input_size 
      embed_input = self.embedding(target)   
      decode_output, h_t = self.rnn(embed_input, h) #embedding 된 단어 vector를 rnn에 넣어줌. 
      return decode_output, h_t
  

In [0]:
class Seq2Seq(nn.Module):
  def __init__(self, vocab_size, embed_size, hidden_size, num_layers, bidirectional=False):
    super(Seq2Seq, self).__init__()
    self.hidden_size = hidden_size
    
    self.encoder = Encoder(vocab_size, embed_size, hidden_size, num_layers)
    self.decoder = Decoder(vocab_size, embed_size, hidden_size, num_layers)
    self.output_layer = nn.Sequential(nn.Linear(hidden_size, vocab_size), nn.ReLU())


   
  def forward(self, inputs, h_0):
      source, target = inputs
      
      encoder_output, h_t = self.encoder(source, h_0)
      decoder_output, h_t = self.decoder(target, h_t)
      decoder_output = decoder_output[:, :-1, :]  # [batch_size x seq_len, hidden] : for 문을 돌아야함 
      
      flatten_output = decoder_output.reshape(-1, self.hidden_size # [batch_size * seq_len, hidden] : 2차원 , rnn 돌릴때 for 문을 안돌고 matrix 상에서 계산할 수 있음.
      output = self.output_layer(flatten_output)
      
      return output

In [0]:
dataset = SequenceData(src_train, trg_train)
dataloader = DataLoader(dataset, 128, shuffle=False, num_workers=0, collate_fn=collate_fn)

vocab_size = 10004
embedding_size = 300
hidden_size = 300
num_layers =2
model = Seq2Seq(vocab_size, enbedding_size, hidden_size, num_layers, False).cuda()

learning_rate = 0.001
criterion = nn.CrossEntropyLoss(ignore_index=src_w2i['<PAD>'], reduction='sum')
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(100):
  for step, (src, trg) in enumerate(dataloader):
    model.zero_grad()
    
    trg_input = [[trg_w2i['<SOS>']] + t for t in trg]
    scr = torch.tensor(src, dtype=torch.long).cuda()
    trg_input = torch.tensor(trg_input, dtype=torch.long).cuda()
    h_0 = torch.zeros(num_layers, len(src), hidden_size).cuda()
    
    output = mode((src, trg_input), h_0)
    
    trg = torch.tensor(trg, dtype=torch.long).cuda().reshape(-1)
    
    loss = criterion(output, trg) / src.shape[0]
    loss.backward()
    optimizer.step()
    
    if step % 50 ==0:
      print("Epoch: {:3,} Step: {:10,} Loss: {:.3f}".formate(epoch, step, loss))
      
  # epoch가 10일 때의 learning_rate = 0.001 * (0.9)**(11) =  0.001 * 0.3xx = 0.0003xx 
  # epoch 돌수록 learning rate 이 줄어듬
  for param_group in optimizer.param_groups:
    param_group['lr'] = learning_rate * (0.9)**(epoch+1)